In [22]:
import json
import torchvision.transforms as transforms
from tqdm import tqdm
import numpy as np
from PIL import Image

In [23]:
##### 1. LPIPS
import sys
sys.path.insert(0,'../../../ShiftTolerant-LPIPS')
import stlpips
# model = stlpips.LPIPS(net='alex') # author
# model = stlpips.LPIPS(net='alex', variant="vanilla")
# model = stlpips.LPIPS(net='alex', variant="shift_tolerant")
# model = stlpips.LPIPS(net='vgg') # author
# model = stlpips.LPIPS(net='vgg', variant="vanilla")
model = stlpips.LPIPS(net='vgg', variant="shift_tolerant")
# model = stlpips.DSSIM(use_gpu=0,colorspace="RGB")

load_size = 256
transform_list = []
transform_list.append(transforms.Scale(load_size))
transform_list += [transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5))]
transform = transforms.Compose(transform_list)

def load_data(im):
    l_img = Image.open(im).convert('RGB').crop((0,0,256,256))
    l_img = transform(l_img).unsqueeze(0)
    r_img = Image.open(im).convert('RGB').crop((256+20,0,256+20+256,256))
    r_img = transform(r_img).unsqueeze(0)
    return l_img, r_img

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]
Loading model from: /scratch/abhijay/ShiftTolerant-LPIPS/stlpips/weights/vST0.0/vgg_shift_tolerant.pth


In [24]:
# #### 2. DISTS
# from DISTS_pytorch import DISTS
# model = DISTS()

# load_size = 256
# transform_list = []
# transform_list.append(transforms.Scale(load_size))
# transform_list += [transforms.ToTensor()]
# transform = transforms.Compose(transform_list)

# def load_data(im):
#     l_img = Image.open(im).convert('RGB').crop((0,0,256,256))
#     l_img = transform(l_img).unsqueeze(0)
#     r_img = Image.open(im).convert('RGB').crop((256+20,0,256+20+256,256))
#     r_img = transform(r_img).unsqueeze(0)
#     return l_img, r_img

In [25]:
f = open("user_jnd.json","r")
user_jnd = json.load(f)

In [26]:
ds = []
sames = []
for k in tqdm(user_jnd.keys()):
#     im = "/".join(["0_9_pix_shift",k])
    im = k
    responses = np.mean(user_jnd[k])
    sames.append(responses)
    l, r = load_data(im)
    ds.append(model.forward(l,r).item())

100%|██████████| 301/301 [00:39<00:00,  7.67it/s]


In [27]:
# ##### 3. PIM
# with open('pim1_user_jnd_sames.npy', 'rb') as f:
#     sames = np.load(f)
# with open('pim1_user_jnd_ds.npy', 'rb') as f:
#     ds = np.load(f)
# ds = ds.flatten()

In [28]:
sames = np.array(sames)
ds = np.array(ds)

sorted_inds = np.argsort(ds)
ds_sorted = ds[sorted_inds]
sames_sorted = sames[sorted_inds]

TPs = np.cumsum(sames_sorted)
FPs = np.cumsum(1-sames_sorted)
FNs = np.sum(sames_sorted)-TPs

precs = TPs/(TPs+FPs)
recs = TPs/(TPs+FNs)
score = stlpips.voc_ap(recs,precs)

print(score)

0.7747415559648578
